<a href="https://colab.research.google.com/github/Ishar786/Python/blob/main/SWOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install the Google AI library, requests for web access, BeautifulSoup for HTML parsing, and lxml for speed
!pip install -q -U google-generativeai requests beautifulsoup4 lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 50.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.


In [6]:
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import lxml
from google.colab import userdata
import textwrap
from IPython.display import Markdown

# --- CONFIGURATION (No changes) ---
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
except userdata.SecretNotFoundError as e:
    print("API Key not found. Please follow Step 1 to add your GOOGLE_API_KEY to Colab Secrets.")
    raise e
model = genai.GenerativeModel('gemini-1.5-flash')

# --- MODULE 1: WEB SCRAPER (No changes) ---
# ... [Keeping this function the same as before, collapsed for brevity] ...
def get_article_text(url: str) -> str:
    print(f"Scraping URL: {url}...")
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml')
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()
        main_content = soup.find('article') or soup.find('main') or soup.body
        text = main_content.get_text(separator='\n', strip=True)
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        cleaned_text = '\n'.join(chunk for chunk in chunks if chunk)
        print("Scraping successful.")
        return cleaned_text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

# --- MODULE 2: AI ANALYSIS CORE (FINAL TWEAK FOR FLOW) ---
def generate_swot_point(text: str, company_name: str, swot_category: str, example: str) -> str:
    print(f"Analyzing text for {swot_category} regarding {company_name}...")

    # REFINED PROMPT with a final instruction for better narrative flow
    prompt = f"""
    You are a business analyst who MUST follow a very specific writing formula.
    Analyze the provided "Press Release Text" and generate a single paragraph for a SWOT analysis about the company '{company_name}'.

    **YOUR TASK IS TO FOLLOW THIS EXACT 5-STEP FORMULA:**

    **Step 1: The Opening Sentence.** Start with the date (month year), followed by the company name, the action (e.g., 'has acquired', 'has partnered'), the entity involved, and the immediate purpose.

    **Step 2: The Detail Sentence.** In a new sentence, provide a specific, factual detail about the entity mentioned in Step 1.

    **Step 3: The "Would" Benefit Sentence.** In a new sentence, describe the direct benefit of the action using the word "would". **You can use transitional phrases like "Through this acquisition..." to start this sentence for better flow.**

    **Step 4: The Strategic Benefit Sentence.** In another new sentence, describe the broader strategic benefit or how it helps the company's long-term goals. **You can use transitional phrases like "Further..." to start this sentence.**

    **Step 5: The Concluding Sentence.** In a final sentence, link the action to a larger company mission or goal.

    **DO NOT DEVIATE FROM THIS 5-STEP STRUCTURE. The output MUST be a single paragraph.**

    ---
    **PERFECT EXAMPLE TO MIMIC:**
    {example}
    ---

    **Press Release Text:**
    ---
    {text}
    ---
    """

    try:
        response = model.generate_content(prompt)
        print("Analysis complete.")
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred during AI analysis: {e}")
        return None

# --- MODULE 3: SCALABLE SWOT FRAMEWORK (Single, best example) ---
# We will now use ONLY your best example to reinforce the template.
SWOT_EXAMPLES = {
    "Opportunity": textwrap.dedent("""
        in January 2025, Elevance has acquired Indiana University (IU) Health Plans, to enhance its local presence, expanding product offerings. IU Health Plans is a managed care organization created by Indiana University Health which provides Medicare advantage plans to 19,000 people across 36 countries. This acquisition would elevate quality and would expand Elevance's product offerings. This acquisition would strengthen the company’s efforts to cultivate healthier communities and improve health outcomes for those they are privileged to serve. This strategic step is in line with Elevance’s health equity goals and allow it to provide comprehensive access to high-quality care and timely interventions.
    """)
}

# --- Main Runner Function (No changes) ---
def analyze_url_for_swot(url: str, company_name: str, swot_category: str):
    if swot_category not in SWOT_EXAMPLES:
        print(f"Error: SWOT category '{swot_category}' is not defined.")
        return
    # Note: Using the manual text method since scraping this site is blocked.
    article_text = """
    Elevance Health is continuing its dealmaking streak, this time with the acquisition of Indiana University Health's health plan. IU Health Plans offers Medicare Advantage and commercial plans to employers and individuals in the state, and the deal includes its risk-based and medical management services. The acquisition is expected to close later this year, though financial details were not disclosed. "The acquisition of IU Health Plans will help us continue to build on our commitment to expanding access to high-quality, affordable healthcare for people in Indiana," said Felicia Norwood, executive vice president and president of government health benefits for Elevance Health, in a statement. "We look forward to the opportunity to leverage our resources and broad networks to provide IU Health Plan members with a high-value, whole-health experience." Elevance said it will work with IU Health to ensure a smooth transition for members, who will be able to continue to access IU Health's provider network. Elevance's own provider network will become available to members following the acquisition. Elevance has been on an M&A tear of late, most recently closing its acquisition of specialty pharmacy BioPlus. It also snapped up Blue Cross and Blue Shield of Louisiana in a $2.5 billion deal. The insurer has also been active in the government programs space, acquiring Integra Managed Care in New York.
    """
    example = SWOT_EXAMPLES[swot_category]
    swot_paragraph = generate_swot_point(article_text, company_name, swot_category, example)
    if swot_paragraph:
        print("\n--- Generated SWOT Paragraph ---\n")
        display(Markdown(swot_paragraph))

In [7]:
# --- INPUTS: Paste the article text here ---

# --- ANALYSIS: Change the company name and SWOT category as needed ---

company_name = "Cigna group"
swot_category_to_find = "Opportunity"

# I have copied the text for you from the URL.
article_text_manual = """
Cigna and Singleton Associates, a Radiology Partners-Affiliated Practice, Sign Multi-Year Agreement to Expand Access to Radiology Care Across Texas
April 23, 2025
HOUSTON (April 23, 2025) – Cigna Healthcare (Cigna) and Radiology Partners-affiliated practice, Singleton Associates, P.A. (Singleton), have entered into a new multi-year agreement, effective June 1, 2025. This agreement will create in-network access to Singleton’s robust network of subspecialized radiologists for Cigna’s commercial members across the state.

This agreement marks a major win for the Texas payor and provider communities, exhibiting collaboration and leadership to expand the availability and cost-effectiveness of Singleton’s advanced, technology-enabled imaging services throughout the state. Singleton is known for its subspecialized radiologists, many of whom are fellowship-trained in advanced fields, bringing deep expertise to the patients it serves. As a Radiology Partners-affiliated practice, Singleton radiologists have access to innovative quality programs, support services, advanced technology and artificial intelligence tools to support the delivery of care and ultimately improve patient outcomes: faster reads, better diagnosis and greater access. This is especially critical given the ongoing national physician shortage.

The collaboration also reduces administrative friction in the healthcare system and eliminates unnecessary costs for patients, self-funded employer plans and payors. When Texas payors and providers work together to reach sustainable in-network agreements, self-funded employer groups in Texas are not burdened by the substantial costs of the federal No Surprises Act (NSA) Independent Dispute Resolution (IDR) process, thereby delivering materially lower costs and greater financial predictability for all parties involved. Also, effective immediately, all pending Singleton and Cigna NSA disputes will be settled outside of the NSA IDR process, yielding immediate and significant cost savings for all parties.

“Bringing high-quality radiology care in network doesn’t just improve patient access—it lowers healthcare costs and simplifies the system for everyone,” said Rich Whitney, CEO of Radiology Partners. “By avoiding thousands of unnecessary NSA disputes, we’re helping employers reduce administrative costs and focus resources where they matter most: on patient care. I want to thank the Cigna team for their partnership in bringing this win-win agreement to a successful completion.”

Beginning June 1, 2025, Cigna’s commercial members can take full advantage of their in-network health plan benefits when receiving care from Singleton—unlocking expanded access to differentiated imaging services for the remainder of this benefit year and for years to come.

About Singleton Associates, P.A.
Singleton Associates got its start more than 66 years ago as the first radiology provider to offer 24/7 in-house services to hospitals to make sure patients received the radiological care they needed, when they needed it. Learn more at singletonassociates.radpartners.com.

About Radiology Partners
Radiology Partners, through its owned and affiliated practices, is a leading radiology practice in the U.S., serving more than 3,400 hospitals and other healthcare facilities with high quality radiology, technology and artificial intelligence solutions. As a physician-led and physician-owned practice, our mission is to transform radiology by innovating across clinical value, technology, service and economics, while elevating the role of radiology and radiologists in healthcare. Using a proven healthcare services model, Radiology Partners provides consistent, high-quality care to patients, while delivering enhanced value to the hospitals, clinics, imaging centers and referring physicians we serve. Learn more at radpartners.com and connect with us at @Rad_Partners.


"""

In [8]:
# --- RUN THE ANALYSIS (using the manual text) ---

if swot_category_to_find in SWOT_EXAMPLES:
    examples = SWOT_EXAMPLES[swot_category_to_find]
    # We call the AI function directly, skipping the scraper
    swot_paragraph = generate_swot_point(article_text_manual, company_name, swot_category_to_find, examples)

    if swot_paragraph:
        print("\n--- Generated SWOT Paragraph ---\n")
        display(Markdown(swot_paragraph))
else:
    print(f"Error: SWOT category '{swot_category_to_find}' is not defined.")

Analyzing text for Opportunity regarding Cigna group...
Analysis complete.

--- Generated SWOT Paragraph ---



April 2025, Cigna Group has partnered with Singleton Associates, a Radiology Partners-affiliated practice, to expand access to radiology care in Texas. Singleton Associates is a network of subspecialized radiologists with advanced technology and AI tools, offering faster reads and better diagnoses. Through this partnership, Cigna would provide its commercial members in Texas with in-network access to Singleton's high-quality radiology services. Further, this collaboration reduces administrative costs and avoids unnecessary disputes under the No Surprises Act, leading to cost savings for patients and employers. This strategic alignment with Singleton supports Cigna's mission to improve access to quality, cost-effective healthcare for its members and aligns with their broader goals of enhancing the patient experience and streamlining healthcare delivery.